In [2]:
!pip install requests transformers pandas datetime vaderSentiment requests yfinance finnhub-python
!pip install nltk
!pip install vaderSentiment
!pip install yahoo-fin
!pip install finvizfinance
!pip install "websockets>=11.0"
!pip install yfinance yahoo_fin nltk textblob matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=93dfafd3652968732ed6f6afeea9945fa3b7faa8e89e6badd3f1b5cc69aa88ce
  Stored

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 10.4
    Uninstalling websockets-10.4:
      Successfully uninstalled websockets-10.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 15.0.1 which is incompatible.


INFO: pip is looking at multiple versions of pyppeteer to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of pyppeteer to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyppeteer: filename=pyppeteer-0.0.25-py3-none-any.whl size=78356 sha256=0e2993d9c97274892c49ebd47a3ade08ae284b782aeaed05a5a54097a06f135e
  Stored in directory: /root/.cache/pip/wheels/73/a3/1e/3a15782836222d82b917f1ebf652f9db54eec93f3268a42bcf
Successfully built pyppeteer
  Attempting uninstall: pyppeteer
    Found existing installation: pyppeteer 2.0.0
    Uninstalling pyppeteer-2.0.0:
      Successfully uninstalled pyppeteer-2.0.0


In [4]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
from yahoo_fin import news as yahoo_fin_news
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Setup
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()
tz = pytz.timezone('US/Eastern')
now = datetime.now(tz)
start_window = now - timedelta(days=90)

TICKERS = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "NVDA", "PFE", "MRNA", "DIS", "T"]
INTERVALS = {
    "1 Hour Ago": timedelta(hours=1),
    "4 Hours Ago": timedelta(hours=4),
    "Previous Day Close": timedelta(days=1),
    "7 Days Ago": timedelta(days=7)
}

class NewsAnalyzer:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def fetch_articles(self):
        try:
            return [
                {
                    "Ticker": self.ticker,
                    "Title": art["title"],
                    "Summary": art["summary"],
                    "URL": art["link"],
                    "Datetime": datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz)
                }
                for art in yahoo_fin_news.get_yf_rss(self.ticker)
                if start_window <= datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz) <= now
            ]
        except Exception as e:
            print(f"Error fetching news for {self.ticker}: {e}")
            return []

    def get_price_near(self, timepoint):
        try:
            hist = self.stock.history(
                start=timepoint.strftime('%Y-%m-%d'),
                end=(timepoint + timedelta(days=1)).strftime('%Y-%m-%d'),
                interval='1h'
            )
            if not hist.empty:
                hist['Delta'] = abs(hist.index - timepoint)
                return round(hist.sort_values('Delta').iloc[0]['Close'], 2)
        except:
            return None
        return None

    def analyze_article(self, article):
        base_time = article["Datetime"]
        base_price = self.get_price_near(base_time)
        if base_price is None:
            return None

        prices = {}
        pct_changes = {}
        for label, delta in INTERVALS.items():
            time_check = base_time - delta
            past_price = self.get_price_near(time_check)
            prices[label] = past_price if past_price else 'N/A'
            pct_changes[label + " % Change"] = (
                round(((base_price - past_price) / past_price) * 100, 2) if isinstance(past_price, (int, float)) else 'N/A'
            )

        return {
            **article,
            "News Age (Days)": (now - base_time).days,
            "News Time": base_time,
            "Current Price": base_price,
            **prices,
            **pct_changes,
            "Headline Sentiment": round(TextBlob(article["Title"]).sentiment.polarity, 2),
            "Summary Sentiment": round(vader.polarity_scores(article["Summary"])["compound"], 2)
        }

def run_full_analysis():
    full_dataset = []
    for ticker in TICKERS:
        analyzer = NewsAnalyzer(ticker)
        articles = analyzer.fetch_articles()
        for article in articles:
            result = analyzer.analyze_article(article)
            if result:
                full_dataset.append(result)

    df = pd.DataFrame(full_dataset)
    df.to_csv("refactored_yahoo_sentiment_7daytrack.csv", index=False)
    print("Saved to refactored_yahoo_sentiment_7daytrack.csv")
    return df

# Run it
df_result = run_full_analysis()
print(df_result.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-08 -> 2025-06-09)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-07 -> 2025-06-08)
ERROR:yfinance:$AAPL: possibly delisted; no price data found  (1h 2025-06-07 -> 2025-06-08)
ERROR:yfinance:$AAPL: possibly delisted; no p

Saved to refactored_yahoo_sentiment_7daytrack.csv
  Ticker                                              Title  \
0   AAPL  GameStop earnings, US-China talks, May CPI: Wh...   
1   AAPL  Why Apple is falling behind & how it can pull ...   
2   AAPL  Apple Set For Low-Key WWDC News Cycle After AI...   
3   AAPL              Where Will Apple Stock Be in 5 Years?   
4   AAPL  What Apple needs to address in its AI business...   

                                             Summary  \
0  Market Domination Overtime hosts Julie Hyman a...   
1  Patrick Moorhead, founder and CEO at Moor Insi...   
2  Apple's WWDC 2025 is likely to be a low-key af...   
3  Apple shares have been a monster winner in the...   
4  Apple's (AAPL) 2025 Worldwide Developers Confe...   

                                                 URL  \
0  https://finance.yahoo.com/video/gamestop-earni...   
1  https://finance.yahoo.com/video/why-apple-fall...   
2  https://finance.yahoo.com/m/ea39f77e-61e9-34d1...   
3  https:/